In [188]:
# imports
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 140)
pd.set_option('display.width', 2000)
import sys,os
from IPython.core.debugger import Tracer
from IPython.core.debugger import BdbQuit_excepthook
%matplotlib inline
%load_ext line_profiler
import matplotlib.pyplot as plt
import matplotlib
import nltk
import io
import json
import time
from pandas import ExcelWriter
from ast import literal_eval
import hashlib
import unicodedata
import subprocess
import datetime
m = hashlib.md5()
from nltk.metrics import *
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
import zipfile
import pymongo
import re
import random
from tqdm import tqdm

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# experiment configuration

In [189]:
EXP_NAME = 'compwebq_with_triviaqa'
EVAL_SET = 'dev' # train, dev
WRITE_EVIDENCE = True
ADD_SPLITS_TO_TRAIN = True
PRODUCE_ONLY_SAMPLE = False
EVIDENCE_DIR = '/Users/alontalmor/Documents/dev/datasets/triviaqa/triviaqa-rc/'
LOAD_TRIVIAQA = True

# Loading Dataset

## Dev Set

In [190]:
# comparing web triavia-rc dev number of question to unfiltered number of questions
if EVAL_SET == 'dev':
    rl_input_df = pd.DataFrame()
    #data_dir = '/Users/alontalmor/Dropbox/Apps/WebKB/webkb_dev_data/RL_preproc_data/rl_cascade1_epoch0-8/'
    data_dir = '../data/V2_dev_splitpoints/'
    filename = 'dev.json.zip'
    if filename.find('.json.zip') > -1:
        print(filename)
        with zipfile.ZipFile(data_dir + filename, 'r') as myzip:
            with myzip.open(filename.replace('.zip', '')) as myfile:
                curr_batch = pd.DataFrame(json.load(myfile))
        curr_batch = curr_batch[(curr_batch[['split_part1', 'split_part2']].isnull(
        ) * 1.0).sum(axis=1) == 0]  # removing null values
        curr_batch['traj_id'] = curr_batch['ID'] + curr_batch['comp'] + curr_batch['split_part1'].str.replace(" ","") \
                                + ',' + curr_batch['split_part2'].str.replace(" ","")
        if len(rl_input_df) > 0:
            len_before_filter = len(curr_batch)
            curr_batch = curr_batch[
                ~curr_batch['traj_id'].isin(rl_input_df['traj_id'])]
        curr_batch['filename'] = filename
        rl_input_df = rl_input_df.append(curr_batch, ignore_index=True)

    rl_input_df = rl_input_df.set_index('traj_id')

    # dropping exact duplicate splits
    print('size before drop dups: ' + str(len(rl_input_df)))
    rl_input_df = rl_input_df.drop_duplicates(
        ['ID', 'comp', 'split_part1', 'split_part2'])
    print('size after drop dups: ' + str(len(rl_input_df)))

    dataset_filename = '../../../mturk/compqgen/final/complexwebquestions_V1_1/ComplexWebQuestions_dev'
    with open(dataset_filename + '.json', 'r') as outfile:
        complexwebquestions = pd.DataFrame(json.load(outfile))

    rl_input_df = rl_input_df.merge(
        pd.DataFrame(complexwebquestions)[['answers', 'ID']], on='ID', how='inner')
    rl_input_df.rename(columns={'answers_y': 'answers'}, inplace=True)
    del rl_input_df['answers_x']

dev.json.zip
size before drop dups: 3109
size after drop dups: 3109


## Training Set

In [191]:
if LOAD_TRIVIAQA and EVAL_SET == 'train':
    trivia_qa_dir = '/Users/alontalmor/Documents/dev/datasets/triviaqa/triviaqa-unfiltered/'
    with zipfile.ZipFile(trivia_qa_dir + 'unfiltered-web-train-org.json.zip','r') as myzip:
        with myzip.open('unfiltered-web-train-org.json') as myfile:
            questions = json.load(myfile)
            triviaqa_dataset = pd.DataFrame(questions['Data'])

In [19]:
triviaqa_dataset = triviaqa_formated_answers

In [21]:
questions_triviaqa_format = triviaqa_dataset

In [192]:
if EVAL_SET == 'train':
    rl_input_df = pd.DataFrame()
    #data_dir = '/Users/alontalmor/Dropbox/Apps/WebKB/webkb_dev_data/RL_train_data/rl_cascade1_epoch0-8/'
    # V2 Code
    data_dir = '../data/V2_train_splitpoints/'
    for dirname, dirnames, filenames in os.walk(data_dir):

        # making sure noisy sup is added first (because of the default MIN_REWARD_TRESH values
        #if 'noisy_sup_rewarded.json.zip' in filenames:
        #    filenames.remove('noisy_sup_rewarded.json.zip')
        #    filenames = ['noisy_sup_rewarded.json.zip'] + filenames
        # V2 code
        if 'train.json.zip' in filenames:
            filenames.remove('train.json.zip')
            filenames = ['train.json.zip'] + filenames

        for filename in filenames[0:2]:

            if filename.find('.json.zip')>-1:
                print(filename)
                with zipfile.ZipFile(data_dir + filename,'r') as myzip:
                    with myzip.open(filename.replace('.zip','')) as myfile:
                        curr_batch = pd.DataFrame(json.load(myfile))
                curr_batch = curr_batch[(curr_batch[['split_part1', 'split_part2']].isnull() * 1.0).sum(axis=1) == 0] # removing null values
                # V2 Code
                if len(rl_input_df) > 0:
                    curr_batch = curr_batch[curr_batch['ID'].isin(rl_input_df['ID'])]

                curr_batch['traj_id'] = curr_batch['ID'] + curr_batch['comp'] + curr_batch['split_part1'].str.replace(" ","") \
                                            + ',' + curr_batch['split_part2'].str.replace(" ","")
                if len(rl_input_df)>0:
                    len_before_filter = len(curr_batch)
                    curr_batch = curr_batch[~curr_batch['traj_id'].isin(rl_input_df['traj_id'])]
                curr_batch['filename'] = filename
                rl_input_df = rl_input_df.append(curr_batch, ignore_index=True)

    dataset_filename = '../../../mturk/compqgen/final/complexwebquestions_V1_1/ComplexWebQuestions_train'
    with open(dataset_filename + '.json', 'r') as outfile:
        complexwebquestions = pd.DataFrame(json.load(outfile))

    rl_input_df = rl_input_df.merge(
        pd.DataFrame(complexwebquestions)[['answers', 'ID']], on='ID', how='inner')
    rl_input_df.rename(columns={'answers_y': 'answers'}, inplace=True)
    del rl_input_df['answers_x']        

    rl_input_df = rl_input_df.set_index('traj_id')

    # dropping exact duplicate splits
    print('size before drop dups: ' + str(len(rl_input_df)))
    rl_input_df = rl_input_df.drop_duplicates(['ID', 'comp', 'split_part1', 'split_part2'])
    print('size after drop dups: ' + str(len(rl_input_df)))


In [227]:
# checking overlab between V1 training question and V2
dataset_filename = '../../mturk/compqgen/final/complexwebquestions/ComplexWebQuestions_train'
with open(dataset_filename + '.json', 'r') as outfile:
    complexwebquestions_org = pd.DataFrame(json.load(outfile))

In [231]:
len(rl_input_df[rl_input_df['ID'].isin(complexwebquestions_org['ID'])])

19726

In [232]:
len(rl_input_df)

24673

In [193]:
# !! work only on sampple
if PRODUCE_ONLY_SAMPLE:
    rl_input_df = rl_input_df[0:100]

In [194]:
dataset_filename = '../data/ComplexWebQuestions_' + EVAL_SET
with open(dataset_filename + '.json', 'r') as outfile:
    complexwebquestions_org = pd.DataFrame(json.load(outfile)).set_index('ID')
    
rl_input_df.loc[rl_input_df['comp'] == 'composition','composition_answer'] = \
    list(complexwebquestions_org.loc[rl_input_df[rl_input_df['comp'] == 'composition']['ID'],'composition_answer'].astype(object))

In [195]:
questions = []   
for ind in tqdm(range(len(rl_input_df)), total=len(rl_input_df), ncols=80, desc="scoring"):
    question = rl_input_df.iloc[ind]
    if question['comp'] == 'conjunction':
        q_copied = question.copy()
        questions.append(q_copied)
        if ADD_SPLITS_TO_TRAIN:
            q_copied = question.copy()
            q_copied['question'] = question['split_part1']
            questions.append(q_copied)
            q_copied = question.copy()
            q_copied['question'] = question['split_part2']
            questions.append(q_copied)
    elif question['comp'] == 'composition':
        q_copied = question.copy()
        q_copied['ID']
        questions.append(q_copied)
        if ADD_SPLITS_TO_TRAIN:
            q_copied = question.copy()
            q_copied['question'] = question['split_part1']
            q_copied['answers'] = [{'aliases':[],'answer':question['composition_answer']}]
            questions.append(q_copied)
            q_copied = question.copy()
            q_copied['question'] = question['split_part2']
            q_copied['question'] = q_copied['question'].replace('%composition',question['composition_answer'])
            questions.append(q_copied)
    else:
        q_copied = question.copy()
        questions.append(q_copied)





scoring:   0%|                                         | 0/3109 [00:00<?, ?it/s]

scoring:   7%|█▉                           | 209/3109 [00:00<00:01, 2068.29it/s]

scoring:  14%|████                         | 432/3109 [00:00<00:01, 2149.09it/s]

scoring:  21%|██████▏                      | 661/3109 [00:00<00:01, 2191.82it/s]

scoring:  29%|████████▎                    | 889/3109 [00:00<00:01, 2213.44it/s]

scoring:  36%|██████████                  | 1116/3109 [00:00<00:00, 2223.38it/s]

scoring:  43%|████████████                | 1343/3109 [00:00<00:00, 2230.59it/s]

scoring:  50%|█████████████▉              | 1552/3109 [00:00<00:00, 2209.21it/s]

scoring:  57%|███████████████▉            | 1764/3109 [00:00<00:00, 2197.06it/s]

scoring:  63%|█████████████████▋          | 1969/3109 [00:00<00:00, 2168.25it/s]

scoring:  70%|███████████████████▌        | 2171/3109 [00:01<00:00, 2150.02it/s]

scoring:  76%|█████████████████████▎      | 2372/3109 [00:01<00:00, 2130.84it/s]

scoring:  83%|

In [196]:
questions = pd.DataFrame(questions)
# dropping exact duplicate splits
print('size before drop dups: ' + str(len(questions)))
questions = questions.drop_duplicates(['question'])
print('size after drop dups: ' + str(len(questions)))
questions = questions.reset_index(drop=True)

size before drop dups: 9327
size after drop dups: 8987


# ComplexWebQuestions to TriviaQA format


In [197]:

questions_triviaqa_format = pd.DataFrame()
questions_triviaqa_format['QuestionId'] = questions['ID'] + '_' + questions.index.astype(str)
questions_triviaqa_format['Question'] = questions['question']
questions_triviaqa_format['SearchResults'] = [[] for x in range(len(questions_triviaqa_format))]
questions_triviaqa_format['EntityPages'] = [[] for x in range(len(questions_triviaqa_format))]
questions_triviaqa_format['QuestionSource'] = ''
all_answers = []
for ind,q in questions.iterrows():
    if q['answers'][0]['answer'] == None or q['answers'][0]['answer'] == '':
        all_answers.append(None)
    else:
        filtered_answer = []
        for answer in q['answers']:
            if len(answer['answer']) > 0:
                filtered_answer.append(answer)
                
        triviaqa_formated_answers = {'Aliases':[],'NormalizedAliases':[], \
                                    'NormalizedValue':'', \
                                     'Type':'FreeForm','Value':''}
        triviaqa_formated_answers['Value'] = filtered_answer[0]['answer']
        triviaqa_formated_answers['NormalizedValue'] = ' '.join(word_tokenize(filtered_answer[0]['answer'].lower()))
        for answer in filtered_answer:
            triviaqa_formated_answers['Aliases'] += answer['aliases']
            triviaqa_formated_answers['Aliases'].append(answer['answer'])

        triviaqa_formated_answers['NormalizedAliases'] = \
            [' '.join(word_tokenize(word.lower())) for word in triviaqa_formated_answers['Aliases']]
        all_answers.append(triviaqa_formated_answers)
    
questions_triviaqa_format['Answer'] = all_answers
questions_triviaqa_format = questions_triviaqa_format[questions_triviaqa_format['Answer'].notnull()]  

In [198]:
# combining all the training sets
if LOAD_TRIVIAQA and EVAL_SET == 'train':
    print(len(questions_triviaqa_format))
    print(len(triviaqa_dataset))
    questions_triviaqa_format = questions_triviaqa_format.append(triviaqa_dataset,ignore_index=True)
    print(len(questions_triviaqa_format))

# appending snippets to the questions

## From local mongo

In [199]:
MONGODB_URI = 'mongodb://127.0.0.1:27017/webkb'
mongo_client = pymongo.MongoClient(MONGODB_URI)
db = mongo_client.get_default_database()
SearchCache = db['SearchResults_Cache']

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: get_default_database is deprecated. Use get_database instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [200]:
found = 0
#questions['SearchResults'] = None
questions_triviaqa_format['SearchResults'] = questions_triviaqa_format['SearchResults'].astype(object)
for ind in tqdm(range(len(questions_triviaqa_format)), total=len(questions_triviaqa_format), ncols=80, desc="scoring"):
    question  = questions_triviaqa_format.iloc[ind]
    CacheResults = SearchCache.find(
        {'querystr': question['Question'], "page": 0, "type": 'SCREEN'})
    CacheResults_Count = CacheResults.count()
    if CacheResults_Count>0:
        found += 1
        cahched_item = CacheResults.next()
        questions_triviaqa_format.at[ind,'SearchResults'] = cahched_item['results']



scoring:   0%|                                         | 0/8981 [00:00<?, ?it/s]

scoring:   1%|▎                              | 93/8981 [00:00<00:09, 922.91it/s]

scoring:   2%|▌                             | 186/8981 [00:00<00:09, 922.37it/s]

scoring:   3%|▉                             | 264/8981 [00:00<00:09, 874.81it/s]

scoring:   4%|█▏                            | 354/8981 [00:00<00:09, 881.92it/s]

scoring:   5%|█▍                            | 430/8981 [00:00<00:09, 857.04it/s]

scoring:   6%|█▋                            | 515/8981 [00:00<00:09, 855.14it/s]

scoring:   7%|█▉                            | 596/8981 [00:00<00:09, 848.42it/s]

scoring:   8%|██▎                           | 678/8981 [00:00<00:09, 844.58it/s]

scoring:   8%|██▌                           | 756/8981 [00:00<00:09, 836.40it/s]

scoring:   9%|██▊                           | 838/8981 [00:01<00:09, 834.26it/s]

scoring:  10%|███                           | 923/8981 [00:01<00:09, 835.13it/s]

scoring:  11%|

scoring:  89%|█████████████████████████▊   | 8005/8981 [00:09<00:01, 830.32it/s]

scoring:  90%|██████████████████████████▏  | 8096/8981 [00:09<00:01, 831.13it/s]

scoring:  91%|██████████████████████████▍  | 8186/8981 [00:09<00:00, 831.77it/s]

scoring:  92%|██████████████████████████▋  | 8276/8981 [00:09<00:00, 832.27it/s]

scoring:  93%|███████████████████████████  | 8368/8981 [00:10<00:00, 833.06it/s]

scoring:  94%|███████████████████████████▎ | 8458/8981 [00:10<00:00, 833.47it/s]

scoring:  95%|███████████████████████████▋ | 8556/8981 [00:10<00:00, 834.85it/s]

scoring:  96%|███████████████████████████▉ | 8648/8981 [00:10<00:00, 835.46it/s]

scoring:  97%|████████████████████████████▏| 8739/8981 [00:10<00:00, 835.66it/s]

scoring:  98%|████████████████████████████▌| 8829/8981 [00:10<00:00, 835.33it/s]

scoring:  99%|████████████████████████████▊| 8916/8981 [00:10<00:00, 834.76it/s]

scoring: 100%|█████████████████████████████| 8981/8981 [00:10<00:00, 833.56it/s]

In [201]:
print(found)

questions_triviaqa_format = questions_triviaqa_format[questions_triviaqa_format['SearchResults'].notnull()]

print(len(questions_triviaqa_format))

questions_triviaqa_format = questions_triviaqa_format[questions_triviaqa_format['Answer'].notnull()]  

8163
8986


# Combine evidence from multiple queries

In [101]:
(questions['answers'].isnull()).any()

False

# Building Evidence Files

In [ ]:
print('checking if there exist cases in which there are no answers?')
print(len(questions_triviaqa_format[questions_triviaqa_format['Answer'].agg(lambda x: x['Value']).apply(len)==0]['Answer']))

In [202]:
triviaqa_dict = {}
triviaqa_dict['Data'] = questions_triviaqa_format
triviaqa_dict['Domain'] = 'unfiltered-web'
triviaqa_dict['Split'] = EVAL_SET
triviaqa_dict['VerifiedEval'] = False
triviaqa_dict['Version'] = 1.0

In [203]:
data_df = triviaqa_dict['Data']
data_df = data_df.set_index('QuestionId')
org_data_df = data_df.copy(deep=True)
data_df = data_df.rename(columns = {'SearchResults':'OrgSearchResults'})
data_df['SearchResults'] = None

In [204]:
# create a query to file map

if WRITE_EVIDENCE and not os.path.isdir(EVIDENCE_DIR + 'evidence/'):
    os.mkdir(EVIDENCE_DIR + 'evidence/')

if WRITE_EVIDENCE and not os.path.isdir(EVIDENCE_DIR +'evidence/' + EXP_NAME):
    os.mkdir(EVIDENCE_DIR + 'evidence/' + EXP_NAME)
        
train_file_ind = int(0)
for i in tqdm(range(len(data_df)), total=len(data_df), ncols=80, desc="scoring"):
    question = data_df.iloc[i]
    questionID = data_df.index[i]
    # building 10 text files out of 100 snippets
    SearchResults = []
    files = []
    filenames = []
    file_ind = 0
    google_results = question['OrgSearchResults']
    train_file_ind += 1
    
        
    if WRITE_EVIDENCE and not os.path.isdir(EVIDENCE_DIR + 'evidence/' + EXP_NAME + '/' + str(int(train_file_ind / 100))):
        os.mkdir(EVIDENCE_DIR + 'evidence/' + EXP_NAME + '/' + str(int(train_file_ind / 100)))
        #if train_file_ind % 1000 == 0:
        #    print(EVIDENCE_DIR + 'evidence/' + EXP_NAME + '/' + str(int(train_file_ind / 100)))

    for ind, g in enumerate(google_results):
        file_ind = file_ind % 20
        if len(files) <= file_ind:
            file_name = EXP_NAME + '/' + str(int(train_file_ind / 100)) + "/" + \
                questionID + '_' + str(file_ind) + '.txt'
            SearchResults.append({'Rank':ind, 'Description':g['snippet'],'Title':g['title'],'DisplayUrl':g['url'] , \
                          'Url':g['url'] + file_name.replace('/','_').replace('.txt',''),'Filename':file_name })
            files.append('')
            filenames.append(file_name)

        files[file_ind] += str(
            ind) + '. ' + g['title'] + '\n' + g['snippet'] + '\n'

        file_ind += 1

    # saving files
    if WRITE_EVIDENCE:
        for file_str, file_name in zip(files, filenames):
            with open(EVIDENCE_DIR + 'evidence/' + file_name, 'w') as outfile:
                outfile.write(file_str)

    data_df.at[questionID, 'SearchResults'] = SearchResults



scoring:   0%|                                         | 0/8981 [00:00<?, ?it/s]

scoring:   0%|                               | 26/8981 [00:00<00:36, 243.85it/s]

scoring:   1%|▏                              | 52/8981 [00:00<00:35, 251.24it/s]

scoring:   1%|▎                              | 84/8981 [00:00<00:32, 272.34it/s]

scoring:   1%|▎                             | 108/8981 [00:00<00:33, 262.45it/s]

scoring:   1%|▍                             | 133/8981 [00:00<00:34, 259.35it/s]

scoring:   2%|▌                             | 159/8981 [00:00<00:34, 257.93it/s]

scoring:   2%|▌                             | 182/8981 [00:00<00:34, 253.55it/s]

scoring:   2%|▋                             | 212/8981 [00:00<00:33, 258.73it/s]

scoring:   3%|▊                             | 238/8981 [00:00<00:34, 255.01it/s]

scoring:   3%|▉                             | 273/8981 [00:01<00:32, 264.04it/s]

scoring:   3%|█                             | 309/8981 [00:01<00:31, 272.24it/s]

scoring:   4%|

scoring:  39%|███████████▎                 | 3515/8981 [00:09<00:15, 358.73it/s]

scoring:  40%|███████████▍                 | 3553/8981 [00:09<00:15, 358.84it/s]

scoring:  40%|███████████▌                 | 3593/8981 [00:10<00:15, 359.17it/s]

scoring:  40%|███████████▋                 | 3630/8981 [00:10<00:14, 358.84it/s]

scoring:  41%|███████████▊                 | 3666/8981 [00:10<00:14, 358.60it/s]

scoring:  41%|███████████▉                 | 3706/8981 [00:10<00:14, 358.92it/s]

scoring:  42%|████████████                 | 3744/8981 [00:10<00:14, 359.08it/s]

scoring:  42%|████████████▏                | 3785/8981 [00:10<00:14, 359.53it/s]

scoring:  43%|████████████▎                | 3823/8981 [00:10<00:14, 359.46it/s]

scoring:  43%|████████████▍                | 3861/8981 [00:10<00:14, 359.05it/s]

scoring:  43%|████████████▌                | 3899/8981 [00:10<00:14, 359.24it/s]

scoring:  44%|████████████▋                | 3936/8981 [00:10<00:14, 359.04it/s]

scoring:  44%|██

scoring:  76%|██████████████████████       | 6818/8981 [00:19<00:06, 344.09it/s]

scoring:  76%|██████████████████████       | 6851/8981 [00:19<00:06, 343.71it/s]

scoring:  77%|██████████████████████▏      | 6883/8981 [00:20<00:06, 343.45it/s]

scoring:  77%|██████████████████████▎      | 6914/8981 [00:20<00:06, 343.14it/s]

scoring:  77%|██████████████████████▍      | 6947/8981 [00:20<00:05, 343.05it/s]

scoring:  78%|██████████████████████▌      | 6986/8981 [00:20<00:05, 343.26it/s]

scoring:  78%|██████████████████████▋      | 7028/8981 [00:20<00:05, 343.59it/s]

scoring:  79%|██████████████████████▊      | 7064/8981 [00:20<00:05, 343.65it/s]

scoring:  79%|██████████████████████▉      | 7100/8981 [00:20<00:05, 343.41it/s]

scoring:  79%|███████████████████████      | 7134/8981 [00:20<00:05, 343.23it/s]

scoring:  80%|███████████████████████▏     | 7168/8981 [00:20<00:05, 342.93it/s]

scoring:  80%|███████████████████████▎     | 7204/8981 [00:21<00:05, 342.97it/s]

scoring:  81%|██

# calc answer with snippets percentage

In [ ]:
answer_found_mat = np.zeros((len(data_df),100))
q_ind = 0
for ind,question in data_df.iterrows():
    normalized_aliases = question['Answer']['NormalizedAliases']
    
    for s_ind,result in enumerate(question['SearchResults']):
        for alias in normalized_aliases:
            result_text = result['Title'] + ' ' + result['Description']
            p = re.compile(r'\b({0})\b'.format(alias), re.IGNORECASE)
            res = re.findall(p, result_text)
            #if result_text.find(alias)>-1:
             #   answer_found_mat[q_ind , s_ind] = 1
            if len(res) > 0:
                answer_found_mat[q_ind , s_ind] = 1
                break
            #else:
            #    print(alias + ' --- ' + result_text)
            
            
    q_ind += 1 
    if q_ind % 5000 == 0:
        print(q_ind)

In [ ]:
(answer_found_mat.sum(axis=1)>0).sum() / len(org_data_df)

In [ ]:
len(data_df.iloc[2]['SearchResults'])

# visiaully comparing snippets from google and original TriviaQA

In [ ]:
replaced_inds = pd.DataFrame((org_data_df['SearchResults'] != data_df['SearchResults']).index)
inds = replaced_inds['QuestionId'].sample(n=10)

In [ ]:

for d,ind in inds.iteritems():
    print('--------' + ind + '-------------')
    print('Question:  '  + data_df.loc[ind,'Question'])
    print('Answer:  '  + data_df.loc[ind,'Answer']['NormalizedValue'])
    print('Google:  '  + data_df.loc[ind,'SearchResults'][0]['Description'])
    print('Original Bing:  '  + org_data_df.loc[ind,'SearchResults'][0]['Description'])

# saving 

In [205]:
del data_df['OrgSearchResults']

# randomizing the samples:
data_df = data_df.sample(frac=1)
#name = 'unfiltered-web-train-compwebq_noisy_sup'
if not os.path.isdir(EXP_NAME):
    os.mkdir(EXP_NAME)

if EVAL_SET == 'dev':
    triviaqa_dict['Data'] = data_df.reset_index().to_dict(orient='rows')
    with zipfile.ZipFile(EXP_NAME + '/' + "unfiltered-web-dev.json.zip", "w", zipfile.ZIP_DEFLATED) as zip_file:
        zip_file.writestr('unfiltered-web-dev.json', json.dumps(triviaqa_dict, sort_keys=True, indent=4))
else:
    triviaqa_dict['Data'] = data_df.reset_index().to_dict(orient='rows')
    with zipfile.ZipFile(EXP_NAME + '/' + "unfiltered-web-train.json.zip", "w", zipfile.ZIP_DEFLATED) as zip_file:
        zip_file.writestr('unfiltered-web-train.json', json.dumps(triviaqa_dict, sort_keys=True, indent=4))